# Convert downloaded TCGA datasets into sample × gene matrices

This notebook is updated to include the data from the [TCGA PanCanAtlas April 2018 updates](http://www.cell.com/pb-assets/consortium/pancanceratlas/pancan/index.html).

In [1]:
import collections
import os

import pandas

## Read gene information

In [2]:
# Load genes
path = os.path.join('download', 'genes', 'genes.tsv')
gene_df = (pandas.read_table(path, dtype='str')
    .set_index('entrez_gene_id', drop=False)
    [['entrez_gene_id', 'symbol', 'description', 'gene_type']]
)
gene_df.head(2)

,entrez_gene_id,symbol,description,gene_type
entrez_gene_id,,,,
1,1,A1BG,alpha-1-B glycoprotein,protein-coding
2,2,A2M,alpha-2-macroglobulin,protein-coding


In [3]:
# Load gene updater
path = os.path.join('download', 'genes', 'updater.tsv')
updater_df = pandas.read_table(path)
old_to_new_entrez = dict(zip(updater_df.old_entrez_gene_id,
                             updater_df.new_entrez_gene_id))

In [4]:
# Load chromosome-symbol to entrez_gene_id mapping
path = os.path.join('download', 'genes', 'chromosome-symbol-mapper.tsv')
chr_sym_map_df = pandas.read_table(path)
chr_sym_map_df.chromosome = 'chr' + chr_sym_map_df.chromosome
chr_sym_map_df.head(2)

,symbol,chromosome,entrez_gene_id
0,(FM-3),chr2,10316
1,(IV)-44,chr14,28337


## Read sample information

This file contains sample information. See the [online documentation](https://xenabrowser.net/datapages/?dataset=EB%2B%2BAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.xena&host=https%3A%2F%2Fpancanatlas.xenahubs.net) for more information. For more details on curation refer to [Liu et al. 2018](https://doi.org/10.1016/j.cell.2018.02.052 "An Integrated TCGA Pan-Cancer Clinical Data Resource to Drive High-Quality Survival Outcome Analytics")

In [5]:
path = os.path.join('download', 'diseases.tsv')
disease_df = pandas.read_table(path)
disease_df.head(2)

,acronym,disease
0,ACC,adrenocortical cancer
1,BLCA,bladder urothelial carcinoma


In [6]:
# Data from https://gdc.cancer.gov/resources-tcga-users/tcga-code-tables/sample-type-codes
path = os.path.join('mapping', 'tcga_sampletype_codes.csv')
sampletype_codes_df = pandas.read_csv(path, dtype='str')
sampletype_codes_dict = dict(zip(sampletype_codes_df.Code, sampletype_codes_df.Definition))
sampletype_codes_df.head(2)

,Code,Definition,Short Letter Code
0,01,Primary Solid Tumor,TP
1,02,Recurrent Solid Tumor,TR


In [7]:
path = os.path.join('download', 'Survival_SupplementalTable_S1_20171025_xena_sp.tsv.gz')

# Mapping to rename and filter columns
renamer = collections.OrderedDict([
    ('sample', 'sample_id'),
    ('_PATIENT', 'patient_id'),
    ('cancer type abbreviation', 'acronym'),
    ('__placeholder__', 'disease'),
    ('age_at_initial_pathologic_diagnosis', 'age_diagnosed'),
    ('gender', 'gender'),
    ('race', 'race'),
    ('ajcc_pathologic_tumor_stage', 'ajcc_stage'),
    ('clinical_stage', 'clinical_stage'),
    ('histological_type', 'histological_type'),
    ('histological_grade', 'histological_grade'),
    ('initial_pathologic_dx_year', 'initial_pathologic_dx_year'),
    ('menopause_status', 'menopause_status'),
    ('birth_days_to', 'birth_days_to'),
    ('vital_status', 'vital_status'),
    ('tumor_status', 'tumor_status'),
    ('last_contact_days_to', 'last_contact_days_to'),
    ('death_days_to', 'death_days_to'),
    ('cause_of_death', 'cause_of_death'),
    ('new_tumor_event_type', 'new_tumor_event_type'),
    ('new_tumor_event_site', 'new_tumor_event_site'),
    ('new_tumor_event_site_other', 'new_tumor_event_site_other'),
    ('new_tumor_event_dx_days_to', 'days_recurrence_free'),
    ('treatment_outcome_first_course', 'treatment_outcome_first_course'),
    ('margin_status', 'margin_status'),
    ('residual_tumor', 'residual_tumor'),
    ('_EVENT', 'event_status'),
    ('_TIME_TO_EVENT', 'event_days'),
    ('OS', 'dead'),
    ('OS.time', 'days_survived'),
    ('DSS', 'disease_specific_survival_status'),
    ('DSS.time', 'disease_specific_survival_days'),
    ('DFI', 'disease_free_interval_status'),
    ('DFI.time', 'disease_free_interval_days'),
    ('PFI', 'progression_free_interval_status'),
    ('PFI.time', 'progression_free_interval_days'),
    ('Redaction', 'Redaction')
])

clinmat_df = (
    pandas.read_table(path)
    .rename(columns=renamer)
    .merge(disease_df, how='left')
    [list(renamer.values())]
    .set_index('sample_id', drop=False)
    .sort_values('sample_id')
)

# Fix capitalization of gender and race
clinmat_df.gender = clinmat_df.gender.str.title()
clinmat_df.race = clinmat_df.race.str.title()

# Extract sample-type with the code dictionary
clinmat_df = clinmat_df.assign(sample_type = clinmat_df.sample_id.str[-2:])
clinmat_df.sample_type = clinmat_df.sample_type.replace(sampletype_codes_dict)

In [8]:
clinmat_df.head(2)

,sample_id,patient_id,acronym,disease,age_diagnosed,gender,race,ajcc_stage,clinical_stage,histological_type,...,dead,days_survived,disease_specific_survival_status,disease_specific_survival_days,disease_free_interval_status,disease_free_interval_days,progression_free_interval_status,progression_free_interval_days,Redaction,sample_type
sample_id,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0001-01,TCGA-02-0001-01,TCGA-02-0001,GBM,glioblastoma multiforme,44.0,Female,White,NaN,NaN,Untreated primary (de novo) GBM,...,1.0,358.0,1.0,358.0,NaN,NaN,1.0,137.0,NaN,Primary Solid Tumor
TCGA-02-0003-01,TCGA-02-0003-01,TCGA-02-0003,GBM,glioblastoma multiforme,50.0,Male,White,NaN,NaN,Untreated primary (de novo) GBM,...,1.0,144.0,1.0,144.0,NaN,NaN,1.0,40.0,NaN,Primary Solid Tumor


In [9]:
# What is the distribution of cancer-types
clinmat_df.sample_type.value_counts()

Primary Solid Tumor                                10517
Solid Tissue Normal                                 1413
Metastatic                                           395
Primary Blood Derived Cancer - Peripheral Blood      200
Recurrent Solid Tumor                                 55
Additional - New Primary                              10
Additional Metastatic                                  1
Name: sample_type, dtype: int64

In [10]:
# Save unfiltered dataset to a TSV
path = os.path.join('data', 'complete', 'samples.tsv')
clinmat_df.to_csv(path, sep='\t', float_format='%.0f', index=False)

In [11]:
# Remove "Redacted" tumors
# These patients either withdrew consent or had genome data mismatch errors
clinmat_df = clinmat_df.query('Redaction != "Redacted"').drop("Redaction", axis=1)

# Keep only these sample types
# filters duplicate samples per patient
sample_types = {
    'Primary Solid Tumor',
    'Primary Blood Derived Cancer - Peripheral Blood',
}
clinmat_df.query("sample_type in @sample_types", inplace=True)

In [12]:
# Check that no samples are duplicated
assert not clinmat_df.duplicated('sample_id', keep=False).any()

# Check that all diseases in clinmat_df are in disease_df
assert not set(clinmat_df.acronym) - set(disease_df.acronym)

len(clinmat_df)

10662

In [13]:
clinmat_df.head(2)

,sample_id,patient_id,acronym,disease,age_diagnosed,gender,race,ajcc_stage,clinical_stage,histological_type,...,event_days,dead,days_survived,disease_specific_survival_status,disease_specific_survival_days,disease_free_interval_status,disease_free_interval_days,progression_free_interval_status,progression_free_interval_days,sample_type
sample_id,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0001-01,TCGA-02-0001-01,TCGA-02-0001,GBM,glioblastoma multiforme,44.0,Female,White,NaN,NaN,Untreated primary (de novo) GBM,...,358.0,1.0,358.0,1.0,358.0,NaN,NaN,1.0,137.0,Primary Solid Tumor
TCGA-02-0003-01,TCGA-02-0003-01,TCGA-02-0003,GBM,glioblastoma multiforme,50.0,Male,White,NaN,NaN,Untreated primary (de novo) GBM,...,144.0,1.0,144.0,1.0,144.0,NaN,NaN,1.0,40.0,Primary Solid Tumor


## Read mutation data

This file contains mutation data (which mutations each sample contains). See the [online documentation](https://xenabrowser.net/datapages/?dataset=mc3.v0.2.8.PUBLIC.xena&host=https%3A%2F%2Fpancanatlas.xenahubs.net) for the `MC3` resource. For more information about mutation calling refer to [Ellrott et al. 2018](https://doi.org/10.1016/j.cels.2018.03.002 "Scalable Open Science Approach for Mutation Calling of Tumor Exomes Using Multiple Genomic Pipelines")

Note that duplicate mutation rows, which [occur](https://groups.google.com/d/msg/ucsc-cancer-genomics-browser/eg6nJOFSefw/Z0BM6pU9BAAJ "Message on the Xena Browser Google Group") for samples that were sequenced multiple times, are filtered.

In [14]:
# Load chromosome-symbol to entrez_gene_id mapping
path = os.path.join('download', 'genes', 'chromosome-symbol-mapper.tsv')
chr_sym_map_df = pandas.read_table(path)
chr_sym_map_df.chromosome = 'chr' + chr_sym_map_df.chromosome
chr_sym_map_df.head(2)

,symbol,chromosome,entrez_gene_id
0,(FM-3),chr2,10316
1,(IV)-44,chr14,28337


In [15]:
path = os.path.join('download', 'mc3.v0.2.8.PUBLIC.xena.tsv.gz')
snp_mutation_df = (
    pandas.read_table(path)
    .rename(columns={'sample': 'sample_id', 'gene': 'symbol'})
    .merge(chr_sym_map_df, on='symbol')
    .drop(['chromosome', 'symbol'], axis='columns')
    .drop_duplicates()
)
snp_mutation_df.head(2)

,sample_id,chr,start,end,reference,alt,effect,Amino_Acid_Change,DNA_VAF,SIFT,PolyPhen,entrez_gene_id
0,TCGA-02-0003-01,10,123810032,123810032,C,T,Missense_Mutation,p.T38M,0.88,NaN,benign(0.335),10579
1,TCGA-05-4427-01,10,123846949,123846949,A,T,Missense_Mutation,p.E1645V,0.49,NaN,possibly_damaging(0.489),10579


In [16]:
# Number of samples with at least one mutation
samples_with_mutation_calls = sorted(set(snp_mutation_df.sample_id))
len(samples_with_mutation_calls)

9104

In [17]:
# Number of genes with at least one mutation
snp_mutation_df.entrez_gene_id.nunique()

20393

In [18]:
# Mutations counts by type
snp_mutation_df.effect.value_counts().reset_index()

,index,effect
0,Missense_Mutation,1606302
1,Silent,654079
2,3'UTR,251556
3,Nonsense_Mutation,126716
4,Intron,88756
5,Frame_Shift_Del,77710
6,5'UTR,69267
7,Splice_Site,40180
8,RNA,34116
9,Frame_Shift_Ins,22668


### Convert SNP mutations to gene mutations

The next cell specifies which mutations to preserve as gene-affecting, which were chosen according to the red & blue [mutation effects in Xena](http://xena.ucsc.edu/how-we-characterize-mutations/).

In [19]:
mutations = {
    'Frame_Shift_Del',
    'Frame_Shift_Ins',
    'In_Frame_Del',
    'In_Frame_Ins',
    'Missense_Mutation',
    'Nonsense_Mutation',
    'Nonstop_Mutation',
    'RNA',
    'Splice_Site',
    'Translation_Start_Site',
}

In [20]:
# Mutations effects that were observed but nut included
set(snp_mutation_df.effect.unique()) - mutations

{"3'Flank", "3'UTR", "5'Flank", "5'UTR", 'Intron', 'Silent', 'large deletion'}

In [21]:
gene_mutation_df = (snp_mutation_df
    .query("effect in @mutations")
    .groupby(['sample_id', 'entrez_gene_id'])
    .apply(len)
    .reset_index()
    .rename(columns={0: 'count'})
)

gene_mutation_df.head(2)

,sample_id,entrez_gene_id,count
0,TCGA-02-0003-01,1665,1
1,TCGA-02-0003-01,1956,1


In [22]:
# Create a sample (rows) by gene (columns) matrix of mutation status
gene_mutation_mat_df = (gene_mutation_df
    .pivot_table(index='sample_id',
                 columns='entrez_gene_id',
                 values='count',
                 fill_value=0)
    .reindex(samples_with_mutation_calls, fill_value=0)
    .astype(bool).astype(int)
)
gene_mutation_mat_df.columns = gene_mutation_mat_df.columns.astype(str)
gene_mutation_mat_df.shape

(9104, 20224)

In [23]:
'{:.2%} sample-gene pairs are mutated'.format(
    gene_mutation_mat_df.stack().mean())

'0.90% sample-gene pairs are mutated'

In [24]:
# Save complete mutation matrix
path = os.path.join('data', 'complete', 'mutation-matrix.tsv.bz2')
gene_mutation_mat_df.to_csv(path, sep='\t', compression='bz2')

## Read gene expression data

This file contains gene expression data. See the [online documentation](https://xenabrowser.net/datapages/?dataset=EB%2B%2BAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.xena&host=https%3A%2F%2Fpancanatlas.xenahubs.net) for processing information.

In [25]:
# Read the gene × sample dataset
path = os.path.join('download', 'EB++AdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.xena.tsv.gz')
expr_df = pandas.read_table(path, index_col=0)

In [26]:
# Retrieve symbol to gene mapping for gene expression data
path = os.path.join('mapping', 'HiSeqV2-genes', 'HiSeqV2-gene-map.tsv')
gene_map_df = pandas.read_table(path, dtype='str')
symbol_to_entrez = dict(zip(gene_map_df.symbol, gene_map_df.entrez_gene_id))

# Check for unmapped symbols
unmapped_symbols = set(expr_df.index) - set(symbol_to_entrez)
unmapped_symbols

{'100130426',
 '100133144',
 '100134869',
 '10357',
 '10431',
 '136542',
 '155060',
 '26823',
 '280660',
 '317712',
 '340602',
 '388795',
 '390284',
 '391343',
 '391714',
 '404770',
 '441362',
 '442388',
 '553137',
 '57714',
 '645851',
 '652919',
 '653553',
 '728045',
 '728603',
 '728788',
 '729884',
 '8225',
 '90288'}

In [27]:
# Process the dataset
expr_df = (expr_df
    # Drop genes with NA measurements
    # (See https://github.com/cognoma/cancer-data/issues/40)
    .dropna(axis='rows')
    # Convert gene symbols to entrez gene ids
    .rename(index=symbol_to_entrez)
    .rename(index=old_to_new_entrez)
    # Average expression for rows with the same entrez_gene_id
    .groupby(level=0).mean()
    # Transpose so the data is sample × gene
    .transpose()
    # Sort rows and columns
    .sort_index(axis='rows')
    .sort_index(axis='columns')
)

expr_df.index.rename('sample_id', inplace=True)

# Filter for valid Entrez Genes (This will filter the unmapped symbols)
expr_df = expr_df.loc[:, expr_df.columns.isin(gene_df.entrez_gene_id)]

expr_df.shape

(11069, 16261)

In [28]:
# Number of patients represented in the expression dataset
clinmat_df.query("sample_id in @expr_df.index").patient_id.nunique()

9831

In [29]:
# Peak at the data matrix
expr_df.iloc[:5, :5]

sample,1,100,1000,10000,100009676
sample_id,,,,,
TCGA-02-0047-01,6.98,7.10,11.17,10.34,6.18
TCGA-02-0055-01,8.62,7.80,10.83,9.82,6.60
TCGA-02-2483-01,8.09,8.00,11.50,10.37,6.12
TCGA-02-2485-01,6.41,7.02,12.77,13.31,5.18
TCGA-02-2486-01,6.77,7.69,11.14,9.77,6.24


In [30]:
# Save complete expression matrix
path = os.path.join('data', 'complete', 'expression-matrix.tsv.bz2')
expr_df.to_csv(path, sep='\t', float_format='%.3g', compression='bz2')

## Integrate expression and mutation data

Find samples with both mutation and expression data.

We assume that if a sample was not in the `MC3` data, it was not assayed for mutation ([more info](https://github.com/cognoma/cancer-data/issues/43#issuecomment-380957274)).

In [31]:
sample_ids = sorted(clinmat_df.index & gene_mutation_mat_df.index & expr_df.index)
len(sample_ids)

8397

In [32]:
# Filter expression (x) and mutation (y) matrices for common samples
sample_df = clinmat_df.loc[sample_ids, :]
x_df = expr_df.loc[sample_ids, :]
y_df = gene_mutation_mat_df.loc[sample_ids, :]

In [33]:
# Add a columnn for mutations per sample
sample_df['n_mutations'] = y_df.sum(axis='columns')

In [34]:
x_gene_df = (
    gene_df.loc[x_df.columns, :]
    .assign(mean_expression=x_df.mean(axis='rows'))
    .assign(stdev_expression=x_df.std(axis='rows'))
)
path = os.path.join('data', 'expression-genes.tsv')
x_gene_df.to_csv(path, sep='\t', index=False, float_format='%.4g')
x_gene_df.head(2)

,entrez_gene_id,symbol,description,gene_type,mean_expression,stdev_expression
sample,,,,,,
1,1,A1BG,alpha-1-B glycoprotein,protein-coding,6.540815,2.307743
100,100,ADA,adenosine deaminase,protein-coding,7.444250,1.538879


In [35]:
y_gene_df = (
    gene_df.loc[y_df.columns, :]
    .assign(n_mutations=y_df.sum(axis='rows'))
    .assign(mutation_freq=y_df.mean(axis='rows'))
)
path = os.path.join('data', 'mutation-genes.tsv')
y_gene_df.to_csv(path, sep='\t', index=False, float_format='%.4g')
y_gene_df.head(2)

,entrez_gene_id,symbol,description,gene_type,n_mutations,mutation_freq
entrez_gene_id,,,,,,
1,1,A1BG,alpha-1-B glycoprotein,protein-coding,54,0.006431
2,2,A2M,alpha-2-macroglobulin,protein-coding,198,0.023580


### Cancer type (disease) stats

In [36]:
sample_to_acronym = dict(zip(clinmat_df.sample_id, clinmat_df.acronym))

def get_cancer_count_column(sample_ids):
    """
    sample_ids is a pandas.Series
    """
    sample_ids = pandas.Series(sample_ids)
    acronyms = sample_ids.map(sample_to_acronym)
    counter = collections.Counter(acronyms)
    counts = disease_df.acronym.map(counter)
    return counts.fillna(0).astype(int)

# Compute nubmer of samples per disease (cancer type)
disease_df['n_samples'] = get_cancer_count_column(sample_df.sample_id)
disease_df['n_clinical_samples'] = get_cancer_count_column(clinmat_df.sample_id)
disease_df['n_expression_samples'] = get_cancer_count_column(expr_df.index)
disease_df['n_mutation_samples'] = get_cancer_count_column(gene_mutation_mat_df.index)

# Compute n_mutation summaries for samples in the aligned set
acronyms = list(pandas.Series(y_df.index).map(sample_to_acronym))
groups = y_df.sum(axis='columns').groupby(acronyms)
disease_df['median_mutations'] = disease_df.acronym.map(dict(groups.median()))
disease_df['mean_mutations'] = disease_df.acronym.map(dict(groups.mean()))

# Export to TSV
path = os.path.join('data', 'diseases.tsv')
disease_df.to_csv(path, sep='\t', float_format='%.1f', index=False)

disease_df.head(2)

,acronym,disease,n_samples,n_clinical_samples,n_expression_samples,n_mutation_samples,median_mutations,mean_mutations
0,ACC,adrenocortical cancer,79,92,79,92,27.0,68.000000
1,BLCA,bladder urothelial carcinoma,403,409,405,407,164.0,228.168734


### Export matrices to TSVs

Matrices are saved as sample × gene TSVs. Subsetted matrices are also exported to allow users to quickly explore small portions of the dataset.

In [37]:
path = os.path.join('data', 'samples.tsv')
sample_df.to_csv(path, sep='\t', float_format='%.0f', index=False)

In [38]:
def subset_df(df, nrows=None, ncols=None, row_seed=0, col_seed=0):
    """Randomly subset a dataframe, preserving row and column order."""
    if nrows is None:
        nrows = len(df)
    if ncols is None:
        ncols = len(df.columns)
    return (df
        .sample(n=nrows, random_state=row_seed, axis='rows')
        .sample(n=ncols, random_state=col_seed, axis='columns')
        .sort_index(axis='rows')
        .sort_index(axis='columns')
    )

In [39]:
tsv_args = {'sep': '\t', 'float_format': '%.3g'}

for df, name in (x_df, 'expression-matrix'), (y_df, 'mutation-matrix'):

    # Save full dataset
    path = os.path.join('data', name + '.tsv.bz2')
    df.to_csv(path, **tsv_args, compression='bz2')
    
    # Save subsetted datasets
    for sample, nrows, ncols in ('small', 50, 15), ('all-samples', None, 15), ('all-genes', 50, None):
        path = os.path.join('data', 'subset', '{}-{}.tsv'.format(name, sample))
        subset_df(df, nrows=nrows, ncols=ncols).to_csv(path, **tsv_args)